In [4]:
from hashlib import sha1
import hmac
from wsgiref.handlers import format_date_time
from datetime import datetime
from time import mktime
import base64
import requests
from pprint import pprint
import json

class Auth():

    def __init__(self, app_id, app_key):
        self.app_id = app_id
        self.app_key = app_key

    def get_auth_header(self):
        xdate = format_date_time(mktime(datetime.now().timetuple()))
        hashed = hmac.new(self.app_key.encode('utf8'), ('x-date: ' + xdate).encode('utf8'), sha1)
        signature = base64.b64encode(hashed.digest()).decode()

        authorization = 'hmac username="' + self.app_id + '", ' + \
                        'algorithm="hmac-sha1", ' + \
                        'headers="x-date", ' + \
                        'signature="' + signature + '"'
        return {
            'Authorization': authorization,
            'x-date': format_date_time(mktime(datetime.now().timetuple())),
            'Accept - Encoding': 'gzip'
        }


app_id = '1a7f0642168c470fb3e0a60eb9ac9ab8'
app_key = '0C6NGnwbNLTHvdupL7lAqmmfVQ0'

if __name__ == '__main__':
    a = Auth(app_id, app_key)
    url = "https://ptx.transportdata.tw/MOTC/v2/Rail/Metro/LiveBoard/KRTC?$filter=StationID%20eq%20'O7'&$top=30&$format=JSON"
    response = requests.get(url, headers= a.get_auth_header())
    print(response.text)
    jsondata = json.loads(response.text)
    for data in jsondata:
        print('['+data['StationName']['Zh_tw']+'站]', data['TripHeadSign'], '班車預計再', data['EstimateTime'], '分鐘到達')


[{"LineNO":"O","LineID":"O","LineName":{"Zh_tw":"橘線","En":"Orange Line"},"StationID":"O7","StationName":{"Zh_tw":"文化中心","En":"Cultural Center"},"TripHeadSign":"往大寮","DestinationStaionID":"OT1","DestinationStationName":{"Zh_tw":"大寮","En":"Daliao"},"EstimateTime":0,"SrcUpdateTime":"2019-04-20T19:56:25+08:00","UpdateTime":"2019-04-20T19:57:39+08:00"},{"LineNO":"O","LineID":"O","LineName":{"Zh_tw":"橘線","En":"Orange Line"},"StationID":"O7","StationName":{"Zh_tw":"文化中心","En":"Cultural Center"},"TripHeadSign":"往西子灣","DestinationStaionID":"O1","DestinationStationName":{"Zh_tw":"西子灣","En":"Sizihwan"},"EstimateTime":1,"SrcUpdateTime":"2019-04-20T19:56:25+08:00","UpdateTime":"2019-04-20T19:57:39+08:00"}]
[文化中心站] 往大寮 班車預計再 0 分鐘到達
[文化中心站] 往西子灣 班車預計再 1 分鐘到達
